In [ ]:
import sys
sys.path.insert(0, '../..') 

import torch
import torch.sparse

from src.lorsa_function import topk_lorsa, topk_lorsa_torch_flash, topk_lorsa_custom_flash, topk_lorsa_custom_flash_sparse_decode
from src.utils.compare import compare_reconstruct, compare_reconstruct_grad, compare_latency
from src.utils.lorsa import generate_lorsa_data, lorsa_fwd_bwd
from src.config import LorsaTestConfig

device='cuda'
dtype=torch.float32

In [ ]:
config = LorsaTestConfig(
    device=device,
    dtype=dtype,
    batch_size=4,
    seq_len=2048,
    d_model=4096,
    n_qk_heads=128,
    d_qk_head=128,
    expansion_factor=8,
    topk=128,
    causal=True,
)

In [ ]:
fwd_func_1 = topk_lorsa_custom_flash
fwd_func_2 = topk_lorsa_custom_flash_sparse_decode
fwd_func_3 = topk_lorsa

In [ ]:
compare_reconstruct(
    gen_data_func=generate_lorsa_data,
    fwd_func_1=fwd_func_1,
    fwd_func_2=fwd_func_3,
    config=config,
    atol=6e-3,
    rtol=6e-3,
)
compare_reconstruct(
    gen_data_func=generate_lorsa_data,
    fwd_func_1=fwd_func_2,
    fwd_func_2=fwd_func_3,
    config=config,
    atol=6e-3,
    rtol=6e-3,
)

In [ ]:
compare_reconstruct_grad(
    gen_data_func=generate_lorsa_data,
    fwd_func_1=fwd_func_1,
    fwd_func_2=fwd_func_2,
    fwd_bwd_func=lorsa_fwd_bwd,
    config=config,
    atol=6e-3,
    rtol=6e-3,
)

In [ ]:
compare_latency(
    gen_data_func=generate_lorsa_data,
    fwd_func_1=fwd_func_1,
    fwd_func_2=fwd_func_2,
    fwd_bwd_func=lorsa_fwd_bwd,
    config=config,
)